# Testing all models



Metrics: LM loss, Rouge-1, Rouge-2, BERT-score, self Rouge 1, self similarity, syllable count per line, syllable difference per line

In [1]:
from transformers import BartTokenizer, BartModel,BartForConditionalGeneration
from torch.distributions import Categorical
import pandas as pd
import torch.nn as nn
from undecorated import undecorated
import numpy as np
import evaluate
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset,DataLoader
from transformers import  Seq2SeqTrainingArguments, Seq2SeqTrainer,DataCollatorForSeq2Seq
import torch.optim as optim
import torchvision.utils as vutils
import types
import matplotlib.pyplot as plt
from torch.distributions import Categorical
import random
from collections import Counter
from nltk.tokenize import SyllableTokenizer
import ast

In [2]:
from utils import PoemDataset,encode_sentences_wsyl,encode_sentences

In [3]:
from gan_utils import Generator

In [7]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

In [18]:
rouge = evaluate.load('rouge')
bert_score = evaluate.load('bertscore')

In [6]:
test_generator =  Generator().cuda()#BartForConditionalGeneration.from_pretrained('facebook/bart-base').cuda()
#model = TheModelClass(*args, **kwargs)
test_generator.load_state_dict(torch.load('/home/alexander/nlg-project/GAN_models_finetuned/' + 'four_it_ft_generator_bad.pt')['model_state_dict'])



<All keys matched successfully>

In [15]:
inputs = tokenizer("study", return_tensors="pt")

In [16]:
tokenizer.batch_decode(test_generator.model.eval().cpu().generate(inputs['input_ids'],attention_mask=inputs["attention_mask"],max_length=32, use_cache=True,
        decoder_start_token_id = tokenizer.pad_token_id,
        num_beams= 100,
        early_stopping = True))

['<pad><s> need to study\nand return the power\nof my day</s>']

In [21]:
tokenizer.batch_decode(test_generator.model.eval().cpu().generate(inputs['input_ids'],attention_mask=inputs["attention_mask"],max_length=32, use_cache=True,
        decoder_start_token_id = tokenizer.pad_token_id,
        num_beams= 100,
        early_stopping = True))




["<pad><s>'s the romantic of a whore\nthis summer whenever you have a\nthis financial system of societies that would have you if I had to settle for</s>"]

In [68]:
gen_finetuned = BartForConditionalGeneration.from_pretrained('/home/alexander/nlg-project/BART-base/' + 'checkpoint-25000')






In [66]:
inputs = tokenizer("happy", return_tensors="pt")

In [67]:
tokenizer.batch_decode(gen_finetuned.eval().cpu().generate(inputs['input_ids'],attention_mask=inputs["attention_mask"],max_length=32, use_cache=True,
        decoder_start_token_id = tokenizer.pad_token_id,
        num_beams= 100,
        early_stopping = True))


['<pad><s> happy new year to\nyou and your family i\nlove you so so much</s>']

In [4]:
def syllable_vec(row):
    return [row['line_0_scount'],row['line_1_scount'],row['line_2_scount']]

def change_title(row):
    #adds syllable info to title
    #print(row['Title'])
    return str(row['syllables']) + ';' + str(row['Title'])

def add_syllables_title(part_df, match_df):
    merged = pd.merge(part_df, match_df,how='left', left_on=['Unnamed: 0'], right_index=True)
    merged['syllables'] = merged.apply(syllable_vec,axis=1)
    merged.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y','line_0','line_1','line_2','source','valid','line_0_scount',
                        'line_1_scount','line_2_scount'],inplace=True)
    merged['Title'] = merged.apply(change_title,axis=1)
    
    return merged

In [8]:
processed_data_dir =  '/home/alexander/nlg-project/data/'
test_df = pd.read_csv(processed_data_dir + 'test_data.csv')
whole_df = pd.read_csv(processed_data_dir + 'kaggle_data.csv')
test_df = add_syllables_title(test_df,whole_df)
test_model_syllables = encode_sentences_wsyl(tokenizer,test_df)

In [9]:
processed_data_dir =  '/home/alexander/nlg-project/data/'
test_df = pd.read_csv(processed_data_dir + 'test_data.csv')
whole_df = pd.read_csv(processed_data_dir + 'kaggle_data.csv')
test_df = add_syllables_title(test_df,whole_df)
test_model_syllables = encode_sentences_wsyl(tokenizer,test_df)

In [10]:

test_df = pd.read_csv(processed_data_dir + 'test_data.csv')
test_model_plain = encode_sentences(tokenizer,test_df)
#test_model_plain_mle = encode_sentences_mle(tokenizer,test_df)

In [11]:
    
class PoemDataset(Dataset):
    def __init__(self, df):
        self.labels = df['labels']
        self.mask = df['attention_mask']
        self.input = df['input_ids']
        self.decoder_mask = df['decoder_attention_mask']
        #self.target_transform = target_transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        
        input_ids = self.input[idx]
        attention_masks = self.mask[idx]
        target_ids = self.labels[idx]
        decoder_masks = self.decoder_mask[idx  ]             
        batch = {
          "input_ids": input_ids,
         # "decoder_attention_mask": decoder_masks,
          "attention_mask": attention_masks,
          "label_ids": target_ids,
        }
        return batch
    

In [12]:
eval_ds_syllables = PoemDataset(test_model_syllables)
eval_ds_plain = PoemDataset(test_model_plain)
#eval_ds_plain_mle = PoemDataset(test_model_plain_mle)

In [116]:
eval_ds_plain.__getitem__(0)['label_ids']

tensor([    0,   608,    24, 47320, 50118,  5016,    24,     7,  2841,  2841,
          608, 50118,   405,   473,    24,     7,  2841,     2,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1])

In [113]:
eval_ds_plain.__getitem__(0)['decoder_attention_mask']

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [13]:
class test_generator():
    #tests the generator on several metrics
    def __init__(self,path,ds):
        test_generator =  Generator().cuda()#BartForConditionalGeneration.from_pretrained('facebook/bart-base').cuda()
        #model = TheModelClass(*args, **kwargs)
        test_generator.load_state_dict(torch.load(path)['model_state_dict'])

        self.ds = ds
    
        self.model = test_generator.model
        batch_size = 16 

        args = Seq2SeqTrainingArguments(
            "Gen-testing",
            evaluation_strategy = "epoch",
            #learning_rate=2e-5,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
           # weight_decay=0.01,
           # save_total_limit=3,
           # num_train_epochs=1,
           # predict_with_generate=True,

            learning_rate=1e-4,
            weight_decay=0.01,
            adam_beta1=0.9,
            adam_beta2=0.999,
            adam_epsilon=1e-6,
            max_grad_norm=1.0,
            num_train_epochs=10,
            lr_scheduler_type="linear",
            warmup_ratio=0.1,
        )
        
        self.trainer = Seq2SeqTrainer(
            model=self.model,
            args=args,
            eval_dataset=ds,
            #use_auth_token=False,

            tokenizer=tokenizer
        )
        
    def generate_samples(self):
        #generates 
        dl = DataLoader(self.ds,batch_size=16)
        self.generations = []
        self.generated_tokens = []
        self.references = []
        
        self.titles = []
        #generate 480 samples
        for i,inputs in enumerate(dl):
          #  print(inputs)
           # inputs = inputs.cuda()
            new_poem = ((self.model.eval().generate(inputs['input_ids'].cuda(),attention_mask=inputs["attention_mask"].cuda(),max_length=32, use_cache=True,
                decoder_start_token_id = tokenizer.pad_token_id, num_beams= 100,early_stopping = True)))
            self.generated_tokens.append(new_poem)
            self.generations.append(tokenizer.batch_decode(new_poem,skip_special_tokens=True))
            
            self.references.append(tokenizer.batch_decode(inputs['label_ids'],skip_special_tokens=True))
            
            self.titles.append(tokenizer.batch_decode(inputs['input_ids'],skip_special_tokens=True))
            if i > 30:
                break
           # print(generated)\
        self.generations = [item for sublist in self.generations for item in sublist] 
        self.references = [item for sublist in self.references for item in sublist] 
        self.generated_tokens = [item for sublist in self.generated_tokens for item in sublist] 
        self.titles = [item for sublist in self.titles for item in sublist] 
       # rouge_out = metric.compute(predictions=self.generations, references=self.references)
       # print(rouge_out)
    
    def rouge(self):
        rouge_out = rouge.compute(predictions=self.generations, references=self.references)
        print(rouge_out)
        
    def bert_score(self):
        bert_out = bert_score.compute(predictions=self.generations, references=self.references,lang='en')
        print("BertScore f1 mean: ",np.mean(bert_out['f1']))
        print("BertScore recall mean: ",np.mean(bert_out['recall']))
        print("BertScore precision mean: ",np.mean(bert_out['precision']))
        
    def auto_rouge(self):
        #computes the mean rouge-1 and rouge-2 between each generation and the restt of the generations
        #quantifies mode collapse 
        self.auto_rouge1s = []
        self.auto_rouge2s = []
        
        for i in range(len(self.generations)):
            
            
            refs = self.generations[:i] + self.generations[i+1:]
            pred = [self.generations[i]] * len(refs)
          #  print(pred)

            rouge_out = rouge.compute(predictions=pred, references=refs)
            self.auto_rouge1s.append(rouge_out['rouge1'])
            self.auto_rouge2s.append(rouge_out['rouge2'])
        print("Auto Rouge-1 mean: ",np.mean(self.auto_rouge1s))
        print("Auto Rouge-2 mean: ",np.mean(self.auto_rouge2s))
    
    def self_similarity(self):
        #computes the mean self-similarity of each generation
        #note this is based on TOKENS and not words
        #similarity is (# max same tokens) (# tokens) where tokens do not inclue start, pad, end, or newline
        bad_tokens = [0,1,2,50118]
        self.auto_similarity = []
        for i in range(len(self.generated_tokens)):
            curr_gen = self.generated_tokens[i].cpu().tolist()
            counts = Counter(curr_gen).most_common()
            max_tokens = -1
            token_count = 0
            for token,c in counts:
                if token not in bad_tokens:
                    if max_tokens == -1:
                        max_tokens = c
                    token_count += c
            self.auto_similarity.append(max_tokens/token_count)
        print("Self similarity mean: ",np.mean(self.auto_similarity))
            
        
    def test_generation(self):
        #tests the model at the standard generation task, returning the huggingface generator loss 
        print(self.trainer.evaluate())
        
    def print_generations(self):
        #prints the first 20 generations and coprresponding titles
        dl = DataLoader(self.ds,batch_size=16)
        generations = []
        titles = []
        references = []
        
        for i,inputs in enumerate(dl):
          #  print(inputs)
           # inputs = inputs.cuda()
            new_poem = ((self.model.eval().generate(inputs['input_ids'].cuda(),attention_mask=inputs["attention_mask"].cuda(),max_length=32, use_cache=True,
                decoder_start_token_id = tokenizer.pad_token_id, num_beams= 100,early_stopping = True)))
            
            generations.append(tokenizer.batch_decode(new_poem,skip_special_tokens=True))
            titles.append(tokenizer.batch_decode(inputs['input_ids'],skip_special_tokens=True))
            references.append(tokenizer.batch_decode(inputs['label_ids'],skip_special_tokens=True))
            if i > 2:
                break
           # print(generated)\
        generations = [item for sublist in generations for item in sublist] 
        references = [item for sublist in references for item in sublist] 
        titles = [item for sublist in titles for item in sublist] 
        print('generations')
        print(generations[0:20])
        print('titles')
        print(titles[0:20])
        print('references')
        print(references[0:20])
        
    def test_syllables_general(self):
        #returns the mean and std of sllables of each line in generation
        #usefull where model is not conditioned on syllables
        self.syllables = {}
        self.syllables[0] = []
        self.syllables[1] = []
        self.syllables[2] = []
        
        for poem in self.generations:
            lines = poem.split('\n')
            SSP = SyllableTokenizer()
            for i,l in enumerate(lines):
                if i < 3:
                    #else too many new lines, bad generation
                    tokens = SSP.tokenize(l.strip())
                 #   print(tokens)
                    self.syllables[i].append(len(tokens))
        for line_num in range(3):
            print('mean syl line ' + str(line_num) + ' : ' + str(np.mean(self.syllables[line_num])))
            print('std syl line ' + str(line_num) + ' : ' + str(np.std(self.syllables[line_num])))
                    
    def test_syllables_specific(self,is_valid):
        #returns the mean diff and std between syllables of generations and input desired syllables 
        #if this is part of the model
        if not is_valid:
            return
        
        print("Testing Specific Syllables")
        self.syllable_diff = {}
        self.syllable_diff[0] = []
        self.syllable_diff[1]  = []
        self.syllable_diff[2]  = []
        #cannot use pregenerated syllables incase some are missing 
        
        for j,poem in enumerate(self.generations):
            lines = poem.split('\n')
            SSP = SyllableTokenizer()

            title = self.titles[j]

            syllables = ast.literal_eval(str(self.titles[j]).split(';')[0].strip())
        #    print(syllables)
            for i,l in enumerate(lines):
                if i < 3:
                    #else too many new lines, bad generation
                    tokens = SSP.tokenize(l.strip())
                 #   print(tokens)
                    self.syllable_diff[i].append(np.abs(syllables[i] - len(tokens)))
        for line_num in range(3):
            print('mean difference syl line ' + str(line_num) + ' : ' + str(np.mean(self.syllable_diff[line_num])))
            print('std difference syl line ' + str(line_num) + ' : ' + str(np.std(self.syllable_diff[line_num])))
                    
        
        
        
        
        
    def test(self,conditioned_syllables=False):
        self.test_generation()
        self.generate_samples()
        self.test_syllables_general()
        self.test_syllables_specific(conditioned_syllables)
        self.rouge()
        self.bert_score()
        self.auto_rouge()
        self.self_similarity()
        self.print_generations()

In [14]:
class test_finetuned():
    #tests the generator on several metrics
    def __init__(self,path,ds):

        self.ds = ds
        self.model = BartForConditionalGeneration.from_pretrained(path).cuda()

        batch_size = 16 

        args = Seq2SeqTrainingArguments(
            "Gen-testing",
            evaluation_strategy = "epoch",
            #learning_rate=2e-5,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
           # weight_decay=0.01,
           # save_total_limit=3,
           # num_train_epochs=1,
           # predict_with_generate=True,

            learning_rate=1e-4,
            weight_decay=0.01,
            adam_beta1=0.9,
            adam_beta2=0.999,
            adam_epsilon=1e-6,
            max_grad_norm=1.0,
            num_train_epochs=10,
            lr_scheduler_type="linear",
            warmup_ratio=0.1,
        )
        
        self.trainer = Seq2SeqTrainer(
            model=self.model,
            args=args,
            eval_dataset=ds,
            #use_auth_token=False,

            tokenizer=tokenizer
        )
        
    def generate_samples(self):
        #generates 
        dl = DataLoader(self.ds,batch_size=16)
        self.generations = []
        self.generated_tokens = []
        self.references = []
        self.titles = []
        #generate 480 samples
        for i,inputs in enumerate(dl):
          #  print(inputs)
           # inputs = inputs.cuda()
            new_poem = ((self.model.eval().generate(inputs['input_ids'].cuda(),attention_mask=inputs["attention_mask"].cuda(),max_length=32, use_cache=True,
                decoder_start_token_id = tokenizer.pad_token_id, num_beams= 100,early_stopping = True)))
            self.generated_tokens.append(new_poem)
            self.generations.append(tokenizer.batch_decode(new_poem,skip_special_tokens=True))
            self.references.append(tokenizer.batch_decode(inputs['label_ids'],skip_special_tokens=True))
            self.titles.append(tokenizer.batch_decode(inputs['input_ids'],skip_special_tokens=True))
            if i > 30:
                break
           # print(generated)\
        self.generations = [item for sublist in self.generations for item in sublist] 
        self.references = [item for sublist in self.references for item in sublist] 
        self.generated_tokens = [item for sublist in self.generated_tokens for item in sublist] 
        self.titles = [item for sublist in self.titles for item in sublist] 
       # rouge_out = metric.compute(predictions=self.generations, references=self.references)
       # print(rouge_out)
    
    def rouge(self):
        rouge_out = rouge.compute(predictions=self.generations, references=self.references)
        print(rouge_out)
        
    def bert_score(self):
        bert_out = bert_score.compute(predictions=self.generations, references=self.references,lang='en')
        print("BertScore f1 mean: ",np.mean(bert_out['f1']))
        print("BertScore recall mean: ",np.mean(bert_out['recall']))
        print("BertScore precision mean: ",np.mean(bert_out['precision']))
        
    def auto_rouge(self):
        #computes the mean rouge-1 and rouge-2 between each generation and the restt of the generations
        #quantifies mode collapse 
        self.auto_rouge1s = []
        self.auto_rouge2s = []
        
        for i in range(len(self.generations)):
            
            
            refs = self.generations[:i] + self.generations[i+1:]
            pred = [self.generations[i]] * len(refs)
          #  print(pred)

            rouge_out = rouge.compute(predictions=pred, references=refs)
            self.auto_rouge1s.append(rouge_out['rouge1'])
            self.auto_rouge2s.append(rouge_out['rouge2'])
        print("Auto Rouge-1 mean: ",np.mean(self.auto_rouge1s))
        print("Auto Rouge-2 mean: ",np.mean(self.auto_rouge2s))
    
    def self_similarity(self):
        #computes the mean self-similarity of each generation
        #note this is based on TOKENS and not words
        #similarity is (# max same tokens) (# tokens) where tokens do not inclue start, pad, end, or newline
        bad_tokens = [0,1,2,50118]
        self.auto_similarity = []
        for i in range(len(self.generated_tokens)):
            curr_gen = self.generated_tokens[i].cpu().tolist()
            counts = Counter(curr_gen).most_common()
            max_tokens = -1
            token_count = 0
            for token,c in counts:
                if token not in bad_tokens:
                    if max_tokens == -1:
                        max_tokens = c
                    token_count += c
            self.auto_similarity.append(max_tokens/token_count)
        print("Self similarity mean: ",np.mean(self.auto_similarity))
            
        
    def test_generation(self):
        #tests the model at the standard generation task, returning the huggingface generator loss 
        print(self.trainer.evaluate())
    def print_generations(self):
        #prints the first 48 generations and coprresponding titles
        dl = DataLoader(self.ds,batch_size=16)
        generations = []
        titles = []
        references = []
        
        for i,inputs in enumerate(dl):
          #  print(inputs)
           # inputs = inputs.cuda()
            new_poem = ((self.model.eval().generate(inputs['input_ids'].cuda(),attention_mask=inputs["attention_mask"].cuda(),max_length=32, use_cache=True,
                decoder_start_token_id = tokenizer.pad_token_id, num_beams= 100,early_stopping = True)))
            
            generations.append(tokenizer.batch_decode(new_poem,skip_special_tokens=True))
            titles.append(tokenizer.batch_decode(inputs['input_ids'],skip_special_tokens=True))
            references.append(tokenizer.batch_decode(inputs['label_ids'],skip_special_tokens=True))
            if i > 2:
                break
           # print(generated)\
        generations = [item for sublist in generations for item in sublist] 
        references = [item for sublist in references for item in sublist] 
        titles = [item for sublist in titles for item in sublist] 
        print('generations')
        print(generations)
        print('titles')
        print(titles)
        print('references')
        print(references)
        
    def test_syllables_general(self):
        #returns the mean and std of sllables of each line in generation
        #usefull where model is not conditioned on syllables
        self.syllables = {}
        self.syllables[0] = []
        self.syllables[1] = []
        self.syllables[2] = []
        
        for poem in self.generations:
            lines = poem.split('\n')
            SSP = SyllableTokenizer()
            for i,l in enumerate(lines):
                if i < 3:
                    #else too many new lines, bad generation
                    tokens = SSP.tokenize(l.strip())
                   # print(tokens)
                    self.syllables[i].append(len(tokens))
        for line_num in range(3):
            print('mean syl line ' + str(line_num) + ' : ' + str(np.mean(self.syllables[line_num])))
            print('std syl line ' + str(line_num) + ' : ' + str(np.std(self.syllables[line_num])))
                    
    def test_syllables_specific(self,is_valid):
        #returns the mean diff and std between syllables of generations and input desired syllables 
        #if this is part of the model
        if not is_valid:
            return
        
        print("Testing Specific Syllables")
        self.syllable_diff = {}
        self.syllable_diff[0] = []
        self.syllable_diff[1]  = []
        self.syllable_diff[2]  = []
        #cannot use pregenerated syllables incase some are missing 
        
        for j,poem in enumerate(self.generations):
            lines = poem.split('\n')
            SSP = SyllableTokenizer()

            title = self.titles[j]

            syllables = ast.literal_eval(str(self.titles[j]).split(';')[0].strip())
           # print(syllables)
            for i,l in enumerate(lines):
                if i < 3:
                    #else too many new lines, bad generation
                    tokens = SSP.tokenize(l.strip())
                 #   print(tokens)
                    self.syllable_diff[i].append(np.abs(syllables[i] - len(tokens)))
        for line_num in range(3):
            print('mean difference syl line ' + str(line_num) + ' : ' + str(np.mean(self.syllable_diff[line_num])))
            print('std difference syl line ' + str(line_num) + ' : ' + str(np.std(self.syllable_diff[line_num])))
                    
        
        

       # rouge_out = metric.compute(predictions=self.generations, references=self.references)
        
    def test(self,conditioned_syllables=False):
        self.test_generation()
        self.generate_samples()
        self.test_syllables_general()
        self.test_syllables_specific(conditioned_syllables)
        self.rouge()
        self.bert_score()
        self.auto_rouge()
        self.self_similarity()
        self.print_generations()

In [63]:
finetuned_generator_tester = test_generator('/home/alexander/nlg-project/GAN_models_syllables/' + 'generator.pt',eval_ds_syllables)
finetuned_generator_tester.test(conditioned_syllables=True)

loading configuration file config.json from cache at /home/alexander/.cache/huggingface/hub/models--facebook--bart-base/snapshots/aadd2ab0ae0c8268c7c9693540e9904811f36177/config.json
Model config BartConfig {
  "_name_or_path": "bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2"

{'eval_loss': 38.80013656616211, 'eval_runtime': 34.3048, 'eval_samples_per_second': 209.942, 'eval_steps_per_second': 13.147}
mean syl line 0 : 8.177734375
std syl line 0 : 1.7448731664345576
mean syl line 1 : nan
std syl line 1 : nan
mean syl line 2 : nan
std syl line 2 : nan
Testing Specific Syllables
mean difference syl line 0 : 2.48046875
std difference syl line 0 : 1.7521218651319426
mean difference syl line 1 : nan
std difference syl line 1 : nan
mean difference syl line 2 : nan
std difference syl line 2 : nan
{'rouge1': 0.0798149296, 'rouge2': 0.0012463363578189897, 'rougeL': 0.0719680996364451, 'rougeLsum': 0.07726495970184319}
BertScore f1 mean:  0.8154970010509714
BertScore recall mean:  0.7951456626178697
BertScore precision mean:  0.837202305556275
Auto Rouge-1 mean:  0.5098428056556263
Auto Rouge-2 mean:  0.4020239624103066
Self similarity mean:  0.19889322916666669
generations
[' not the day when i', ' not the day when i', ' they, that I should', ' not the day when i', '

In [64]:
finetuned_generator_tester = test_generator('/home/alexander/nlg-project/GAN_models_finetuned/' + 'four_it_ft_generator_bad.pt',eval_ds_plain)
finetuned_generator_tester.test()

loading configuration file config.json from cache at /home/alexander/.cache/huggingface/hub/models--facebook--bart-base/snapshots/aadd2ab0ae0c8268c7c9693540e9904811f36177/config.json
Model config BartConfig {
  "_name_or_path": "bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2"

{'eval_loss': 1.9570019245147705, 'eval_runtime': 33.9121, 'eval_samples_per_second': 212.373, 'eval_steps_per_second': 13.299}
mean syl line 0 : 4.181640625
std syl line 0 : 0.9520818285996269
mean syl line 1 : 8.357421875
std syl line 1 : 1.3136838387798961
mean syl line 2 : 4.902152641878669
std syl line 2 : 1.11834693503913
{'rouge1': 0.08712119763567386, 'rouge2': 0.00263164241173313, 'rougeL': 0.0771556778237466, 'rougeLsum': 0.08598374520407628}
BertScore f1 mean:  0.814120375434868
BertScore recall mean:  0.8114235509419814
BertScore precision mean:  0.8170057532843202
Auto Rouge-1 mean:  0.4058214834926023
Auto Rouge-2 mean:  0.24658457844248577
Self similarity mean:  0.11791933381288851
generations
[' tell em i\nused to suck on the power\nof my day wow', ' useless\nravens on the strength of\nless than a day', ' use your\nless than a day to study\ni have neither', ' use your\nless than every day for\na study', ' useless\nravens on the strength of\nless than a day', ' need to s

In [ ]:
finetuned_generator_tester = test_generator('/home/alexander/nlg-project/GAN_models_finetuned/' + '3_it_ft_generator.pt',eval_ds_plain)
finetuned_generator_tester.test()


In [ ]:
finetuned_generator_tester = test_generator('/home/alexander/nlg-project/GAN_models_finetuned/' + '2_half_it_ft_generator.pt',eval_ds_plain)
finetuned_generator_tester.test()

In [65]:
finetuned_generator_tester = test_generator('/home/alexander/nlg-project/GAN_models/' + '6_it_nft_generator.pt',eval_ds_plain)
finetuned_generator_tester.test()


loading configuration file config.json from cache at /home/alexander/.cache/huggingface/hub/models--facebook--bart-base/snapshots/aadd2ab0ae0c8268c7c9693540e9904811f36177/config.json
Model config BartConfig {
  "_name_or_path": "bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2"

{'eval_loss': 15.133199691772461, 'eval_runtime': 36.8634, 'eval_samples_per_second': 195.37, 'eval_steps_per_second': 12.234}
mean syl line 0 : 10.001953125
std syl line 0 : 0.044150994357255134
mean syl line 1 : 19.693359375
std syl line 1 : 1.1582393544080642
mean syl line 2 : 23.044921875
std syl line 2 : 2.8311764604394556
{'rouge1': 0.06531888558462579, 'rouge2': 0.002192512456698312, 'rougeL': 0.0586958234167825, 'rougeLsum': 0.06398191021926698}
BertScore f1 mean:  0.8023670351831242
BertScore recall mean:  0.8045374696375802
BertScore precision mean:  0.8003168099094182
Auto Rouge-1 mean:  0.9387648015320235
Auto Rouge-2 mean:  0.8974922897241098
Self similarity mean:  0.14943353810541304
generations
["'s the romantic of a whore\nthis summer will have you if you have\nthis financial system whenever you have a reasonable\nof wealth whenever you", "'s the romantic of a whore\nthis summer will have you if you have\nthis financial system whenever I have a\nof societies whenever yo

In [ ]:
finetuned_generator_tester = test_generator('/home/alexander/nlg-project/GAN_models/' + '4_it_nft_generator.pt',eval_ds_plain)
finetuned_generator_tester.test()

loading configuration file config.json from cache at /home/alexander/.cache/huggingface/hub/models--facebook--bart-base/snapshots/aadd2ab0ae0c8268c7c9693540e9904811f36177/config.json
Model config BartConfig {
  "_name_or_path": "bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2"

{'eval_loss': 16.44188117980957, 'eval_runtime': 37.6406, 'eval_samples_per_second': 191.336, 'eval_steps_per_second': 11.982}
{'rouge1': 0.04775557712498508, 'rouge2': 0.00016276041666666666, 'rougeL': 0.042772379928534596, 'rougeLsum': 0.04742911599380316}
BertScore f1 mean:  0.781419841805473
BertScore recall mean:  0.7957276809029281
BertScore precision mean:  0.7677300509531051


In [66]:
finetuned_generator_tester = test_generator('/home/alexander/nlg-project/GAN_models/' + 'final_gen_cond_title_generator.pt',eval_ds_plain)
finetuned_generator_tester.test()
#I think this was about 4 epochs


loading configuration file config.json from cache at /home/alexander/.cache/huggingface/hub/models--facebook--bart-base/snapshots/aadd2ab0ae0c8268c7c9693540e9904811f36177/config.json
Model config BartConfig {
  "_name_or_path": "bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2"

{'eval_loss': 25.407241821289062, 'eval_runtime': 37.0857, 'eval_samples_per_second': 194.199, 'eval_steps_per_second': 12.161}
mean syl line 0 : 12.0
std syl line 0 : 0.0
mean syl line 1 : nan
std syl line 1 : nan
mean syl line 2 : nan
std syl line 2 : nan
{'rouge1': 0.0003800197963800905, 'rouge2': 0.0, 'rougeL': 0.0003800197963800905, 'rougeLsum': 0.0003800197963800905}
BertScore f1 mean:  0.754906335962005
BertScore recall mean:  0.7771180191775784
BertScore precision mean:  0.7341110940324143
Auto Rouge-1 mean:  1.0
Auto Rouge-2 mean:  1.0
Self similarity mean:  0.5999999999999999
generations
['T markets markets markets market', 'T markets markets markets market', 'T markets markets markets market', 'T markets markets markets market', 'T markets markets markets market', 'T markets markets markets market', 'T markets markets markets market', 'T markets markets markets market', 'T markets markets markets market', 'T markets markets markets market', 'T markets markets markets market'

In [21]:
base_mle_tester = test_finetuned('/home/alexander/nlg-project/BART-base-syllable-loss/' + 'checkpoint-34000',eval_ds_syllables)
#finetuned_generator_tester.test_generation()
base_mle_tester.test(conditioned_syllables=True)

loading configuration file /home/alexander/nlg-project/BART-base-syllable-loss/checkpoint-34000/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_enco

{'eval_loss': 0.7733350992202759, 'eval_runtime': 34.8668, 'eval_samples_per_second': 206.558, 'eval_steps_per_second': 12.935}
mean syl line 0 : 6.8046875
std syl line 0 : 2.5095399035169277
mean syl line 1 : 9.896484375
std syl line 1 : 3.1478891960138715
mean syl line 2 : 6.954635108481263
std syl line 2 : 2.156571813589745
Testing Specific Syllables
mean difference syl line 0 : 0.419921875
std difference syl line 0 : 0.619848766955686
mean difference syl line 1 : 0.798828125
std difference syl line 1 : 0.8388507028720811
mean difference syl line 2 : 0.7554240631163708
std difference syl line 2 : 1.2192348813539247
{'rouge1': 0.2787391048022977, 'rouge2': 0.0553047204006155, 'rougeL': 0.2197894131748271, 'rougeLsum': 0.2693011237880694}


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /home/alexander/.cache/huggingface/hub/models--roberta-large/snapshots/5069d8a2a32a7df4c69ef9b56348be04152a2341/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.23.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /home/alexander/.c

BertScore f1 mean:  0.8413175583118573
BertScore recall mean:  0.841056794510223
BertScore precision mean:  0.8416823233710602
Auto Rouge-1 mean:  0.10184102972441866
Auto Rouge-2 mean:  0.013868704962074145
Self similarity mean:  0.11734460779423994
generations
[' im actually\nso happy for em and it\ndoesnt make me sad', ' am i the only\none who cannot handle the\nsickness of aller', ' anyone know of\na tornado in mississippi\nasking for a friend', ' moreover you should be writing like a psychologist in love\nwith a lover in love with a lover who loves you as much as you love\na', ' marlon james is the\nbest player in the league and\ni love him so much', ' my professor is\ngoing to sleep for the first\ntime in an hour', ' warriors need a\nteamwork day before they go\nto the nba', ' every time i\nconnect with my dad he kills\nme every night', ' stupidity is\none of the worst things you can\ndo for yourself max', ' i suggest the best\nway to start your day is to\nlove ourselves first', 

In [71]:
finetuned_generator_tester.references

[]

In [22]:
finetuned_generator_tester = test_finetuned('/home/alexander/nlg-project/BART-base-syllable-naive/' + 'checkpoint-32000',eval_ds_syllables)
finetuned_generator_tester.test(conditioned_syllables=True)

loading configuration file /home/alexander/nlg-project/BART-base-syllable-naive/checkpoint-32000/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_enc

{'eval_loss': 0.7790548801422119, 'eval_runtime': 34.1542, 'eval_samples_per_second': 210.867, 'eval_steps_per_second': 13.205}
mean syl line 0 : 6.69140625
std syl line 0 : 2.4008633650128735
mean syl line 1 : 9.931640625
std syl line 1 : 3.2423610411010073
mean syl line 2 : 6.81335952848723
std syl line 2 : 2.129071463126769
Testing Specific Syllables
mean difference syl line 0 : 0.408203125
std difference syl line 0 : 0.6054529987870523
mean difference syl line 1 : 0.791015625
std difference syl line 1 : 0.7913579506177084
mean difference syl line 2 : 0.8781925343811395
std difference syl line 2 : 1.71004655156229
{'rouge1': 0.28364452790163464, 'rouge2': 0.05480439553769917, 'rougeL': 0.22205493560258635, 'rougeLsum': 0.27343089481602645}
BertScore f1 mean:  0.8416935628047213
BertScore recall mean:  0.8417783087352291
BertScore precision mean:  0.8417256660759449
Auto Rouge-1 mean:  0.10691523579973278
Auto Rouge-2 mean:  0.013590834264298877
Self similarity mean:  0.1158809587792

In [75]:
finetuned_generator_tester = test_finetuned('/home/alexander/nlg-project/BART-base/' + 'checkpoint-25000',eval_ds_plain)
finetuned_generator_tester.test()

loading configuration file /home/alexander/nlg-project/BART-base/checkpoint-25000/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": 

{'eval_loss': 0.8116941452026367, 'eval_runtime': 31.4397, 'eval_samples_per_second': 229.074, 'eval_steps_per_second': 14.345}
mean syl line 0 : 6.48046875
std syl line 0 : 1.7023677717442367
mean syl line 1 : 9.73046875
std syl line 1 : 2.321269136759768
mean syl line 2 : 6.564453125
std syl line 2 : 1.605576857293893
{'rouge1': 0.28379626945691616, 'rouge2': 0.05750800428608828, 'rougeL': 0.21681880911650797, 'rougeLsum': 0.27448841387104017}
BertScore f1 mean:  0.8406280169729143
BertScore recall mean:  0.8398150866851211
BertScore precision mean:  0.8415693667484447
Auto Rouge-1 mean:  0.13393355905846266
Auto Rouge-2 mean:  0.03923022589649612
Self similarity mean:  0.115445509352878
generations
[' just because someone\ndoesnt love you doesnt mean\nyou dont love em back', ' you should be writing\nlike an aller dreaming in\nlove with a sickness', ' this tornado is\nmaking me want to go to\nmississippi now', ' you should be writing\nlike a physician in love\nwith a physician', ' ma